# Persistent Conversations with `FlotorchADKSession`

This notebook demonstrates how to create a conversational agent that can remember the context of an ongoing interaction. We achieve this by replacing Google ADK's default in-memory session handling with the persistent `FlotorchADKSession` service.

### Prerequesit
Configure model, API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Objectives:
- Understand the role of a `session_service` in managing conversation state.
- Configure the `Runner` to use `FlotorchADKSession` instead of the default.
- Build a simple agent that can recall information from previous turns in the same conversation.

## 1. Setup and Imports

The following cells handle the installation of dependencies, configuration of API credentials, and the import of all necessary components. The key component for this example is `FlotorchADKSession`.

In [ ]:
# install flotorch adk package
%pip install flotorch[adk]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"

In [ ]:
# Import necessary libraries
from flotorch.adk.llm import FlotorchADKLLM
from flotorch.adk.sessions import FlotorchADKSession
from google.adk.agents import LlmAgent
from google.adk import Runner
from google.genai import types


print("Environment setup completed successfully.")

## 2. Configuring the LLM and Agent

First, we configure the `FlotorchADKLLM` to serve as the agent's brain. Then, we create a simple `LlmAgent` with instructions to be a helpful assistant. No special tools are needed for this example, as we are focusing purely on conversational memory.

In [ ]:
model = FlotorchADKLLM(
    model_id = "flotorch/gpt-4o-mini",
    api_key = FLOTORCH_API_KEY,
    base_url = FLOTORCH_BASE_URL,
)
print(f"Flotorch LLM model configured: {model.model}")

In [ ]:
agent = LlmAgent(
    name="Flotorch_Assistant",
    description="A helpful Flotorch AI assistant",
    instruction="You are Flotorch AI assistant. Be helpful and friendly.",
    model=model,
)
print(f"Agent created successfully: {agent.name}")

## 3. Integrating Flotorch Session Management

This is the core of this notebook. We initialize `FlotorchADKSession` using our API credentials. Then, when we create the `Runner`, we pass this `session_service` instance to it. This directs the `Runner` to use the Flotorch backend for storing and retrieving conversation history, enabling stateful interactions.

In [ ]:
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)
print("Runner configured successfully with Flotorch session service.")

## 4. Interactive Chat with Session Memory

The following cells demonstrate the agent's memory. We will start a new session, tell the agent our name, and then ask it to recall the name in a subsequent query. Because we are using `FlotorchADKSession`, the agent will successfully remember the context.

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that request."

async def chat_with_agent(query,session_id):
    response = run_single_turn(query, session_id, USER_ID, runner)
    return response

print("Chat helper functions are ready.")

In [ ]:
# Create a new session for the conversation
session = await runner.session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
)
print(f"Session created with id: {session.id}")

# First turn: Tell the agent your name
response1 = await chat_with_agent("Hello, my name is John.", session.id)
print(f"\nQuery: Hello, my name is John.")
print(f"Response: {response1}")

In [ ]:
# Second turn: Ask the agent to recall your name
response2 = await chat_with_agent("Do you know my name?", session.id)
print(f"Query: Do you know my name?")
print(f"Response: {response2}")

## Summary

This notebook successfully demonstrated how to give a Google ADK agent persistent conversational memory using the Flotorch platform. By swapping the default session service with `FlotorchADKSession`, we enabled the agent to maintain context throughout an interaction.

### Key Achievements:

- **Stateful Conversations**: We configured the `Runner` to use `FlotorchADKSession`, transforming the agent from a stateless to a stateful system.
- **Context Recall**: The agent successfully recalled information (the user's name) from a previous turn within the same session.
- **Robust Foundation**: This pattern provides a solid foundation for building more complex applications like chatbots and customer service agents that require a deep understanding of the ongoing conversation.